# TSFS12 Hand-in exercise 1: Discrete planning in structured road networks
Erik Frisk (erik.frisk@liu.se)

Do initial imports of packages needed

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from misc import Timer, LatLongDistance
from queues import FIFO, LIFO, PriorityQueue
from osm import loadOSMmap

Activate plots in external windows (needed for mission definition)

In [2]:
%matplotlib

Using matplotlib backend: TkAgg


# Load map

In [3]:
dataDir = '../Maps/'
osmFile = 'linkoping.osm'
figFile = 'linkoping.png'
osmMap = loadOSMmap(dataDir + osmFile, dataDir + figFile)

num_nodes = len(osmMap.nodes)  # Number of nodes in the map

Define function to compute possible next nodes from node x and the corresponding distances distances.

In [4]:
def f_next(x):
    """Compute, neighbours for a node"""
    cx = osmMap.distancematrix[x, :].tocoo()
    return cx.col, np.full(cx.col.shape, np.nan), cx.data

# Display basic information about map

Print some basic map information

In [5]:
osmMap.info()

OSMFile: ../Maps/linkoping.osm
Number of nodes: 12112
Number of roads: 2977
Map bounds
  Lon: 15.572100 - 15.650100
  Lat: 58.391000 - 58.420200
  Size: 4545.8 x 3246.9 (m)
Figure file: ../Maps/linkoping.png
  Size: 2296 x 1637 (px)


Plot the map

In [6]:
plt.figure(10)
osmMap.plotmap()
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Linköping');

Which nodes are neighbors to node with index 110?

In [7]:
n, _, d = f_next(110)
print(f'Neighbours: {n}')
print(f'Distances: {d}')

Neighbours: [3400 3401]
Distances: [8.51886025 8.7868951 ]


Look up the distance (in meters) between the nodes 110 and 3400 in the distance matrix?

In [8]:
print(osmMap.distancematrix[110, 3400])

8.518860245357462


Latitude and longitude of node 110

In [9]:
p = osmMap.nodeposition[110]
print(f'Longitude = {p[0]:.3f}, Latitude = {p[1]:.3f}')

Longitude = 15.574, Latitude = 58.398


Plot the distance matrix and illustrate sparseness

In [10]:
plt.figure(20)
plt.spy(osmMap.distancematrix>0, markersize=0.5)
plt.xlabel('Node index')
plt.ylabel('Node index')
density = np.sum(osmMap.distancematrix>0)/num_nodes**2
_ = plt.title(f'Density {density*100:.2f}%')

# Define planning mission

In the map, click on start and goal positions to define a mission. Try different missions, ranging from easy to more complex. 

An easy mission is a mission in the city centre; while a more difficult could be from Vallastaden to Tannefors.

In [11]:
plt.figure(30, clear=True)
osmMap.plotmap()
plt.title('Linköping - click in map to define mission')
mission = {}

mission['start'] = osmMap.getmapposition()
plt.plot(mission['start']['pos'][0], mission['start']['pos'][1], 'bx')
mission['goal'] = osmMap.getmapposition()
plt.plot(mission['goal']['pos'][0], mission['goal']['pos'][1], 'bx')

plt.xlabel('Longitude')
plt.ylabel('Latitude')

print('Mission: Go from node %d ' % (mission['start']['id']), end='')
if mission['start']['name'] != '':
    print('(' + mission['start']['name'] + ')', end='')
print(' to node %d ' % (mission['goal']['id']), end='')
if mission['goal']['name'] != '':
    print('(' + mission['goal']['name'] + ')', end='')
print('')

Mission: Go from node 10917 (Fysikgränd) to node 1063 (Skördegatan)


In [12]:
mission

{'start': {'id': 10917,
  'pos': (15.582793548387098, 58.39286282790258),
  'name': 'Fysikgränd'},
 'goal': {'id': 1063,
  'pos': (15.641922580645161, 58.410164290648105),
  'name': 'Skördegatan'}}

Some pre-defined missions to experiment with. To use the first pre-defined mission, call the planner with
```planner(num_nodes, pre_mission[0], f_next, cost_to_go)```.

In [13]:
pre_mission = [
    {'start': {'id': 10906}, 'goal': {'id': 1024}},
    {'start': {'id': 3987}, 'goal': {'id': 4724}},
    {'start': {'id': 423}, 'goal': {'id': 364}}]
# mission = pre_mission[0]

# Implement planners

In [14]:
def DepthFirst(num_nodes, mission, f_next, heuristic=None, num_controls=0):
    """Depth first planner."""
    t = Timer()
    t.tic()
    
    unvis_node = -1
    previous = np.full(num_nodes, dtype=np.int, fill_value=unvis_node)
    cost_to_come = np.zeros(num_nodes)
    control_to_come = np.zeros((num_nodes, num_controls), dtype=np.int)

    startNode = mission['start']['id']
    goalNode = mission['goal']['id']

    q = LIFO()
    q.insert(startNode)
    foundPlan = False

    while not q.IsEmpty():
        x = q.pop()
        if x == goalNode:
            foundPlan = True
            break
        neighbours, u, d = f_next(x)
        for xi, ui, di in zip(neighbours, u, d):
            if previous[xi] == unvis_node:
                previous[xi] = x
                q.insert(xi)
                cost_to_come[xi] = cost_to_come[x] + di
                if num_controls > 0:
                    control_to_come[xi] = ui

    # Recreate the plan by traversing previous from goal node
    if not foundPlan:
        return []
    else:
        plan = [goalNode]
        length = cost_to_come[goalNode]
        control = []
        while plan[0] != startNode:
            if num_controls > 0:
                control.insert(0, control_to_come[plan[0]])
            plan.insert(0, previous[plan[0]])

        return {'plan': plan,
                'length': length,
                'num_visited_nodes': np.sum(previous != unvis_node),
                'name': 'DepthFirst',
                'time': t.toc(),
                'control': control,
                'visited_nodes': previous[previous != unvis_node]}

## Planning example using the DepthFirst planner

Make a plan using the ```DepthFirst``` planner

In [18]:
df_plan = DepthFirst(num_nodes, mission, f_next)
print('{:.1f} m, {} visited nodes, planning time {:.1f} msek'.format(
    df_plan['length'],
    df_plan['num_visited_nodes'],
    df_plan['time']*1e3))

24191.4 m, 2475 visited nodes, planning time 399.4 msek


Plot the resulting plan

In [21]:
plt.figure(40, clear=True)
osmMap.plotmap()
osmMap.plotplan(df_plan['plan'], 'b',
                label=f"Depth first ({df_plan['length']:.1f} m)")
plt.title('Linköping')
_ = plt.legend()

Plot nodes visited during search

In [23]:
plt.figure(41, clear=True)
osmMap.plotmap()
osmMap.plotplan(df_plan['visited_nodes'], 'b.')
plt.ylabel('Latitude')
plt.xlabel('Longitude')
_ = plt.title('Nodes visited during DepthFirst search')

Names of roads along the plan ...

In [24]:
planWayNames = osmMap.getplanwaynames(df_plan['plan'])
print('Start: ', end='')
for w in planWayNames[:-1]:
    print(w + ' -- ', end='')
print('Goal: ' + planWayNames[-1])

Start: Fysikgränd -- Hannes Alfvéns gata -- Johannes Magnus väg -- Lärdomsgatan -- Nanna Svartz gata -- Poesigränd -- Fredrik Hasselqvists gata -- Lärdomsgatan -- Hugo Theorells gata -- Nelly Sachs gata -- Medicingränd -- Selma Lagerlöfs gata -- Johannes Magnus väg -- Stratomtavägen -- Vallarondellen -- Universitetsvägen -- Vallarondellen -- Rydsvägen -- Vallarondellen -- Malmslättsvägen -- Östgötagatan -- Västra vägen -- Folkungagatan -- Gärdesgatan -- Västanågatan -- Bjälbogatan -- Ulvåsavägen -- Kindagatan -- Skärkindsgatan -- Vifolkagatan -- Ydregatan -- Egnahemsgatan -- Stenbrötsgatan -- Vretagatan -- Kagagatan -- Vretagatan -- Tornhagsvägen -- Örngatan -- Orrgatan -- Vretagatan -- Blomgatan -- Grenadjärgatan -- Industrigatan -- Bergsvägen -- Bergsrondellen -- Bergsvägen -- Industrigatan -- Mellangatan -- Industrigatan -- Steningerondellen -- Industrigatan -- Järnvägsgatan -- Industrigatan -- Steningerondellen -- Industrigatan -- Nordengatan -- Industrigatan -- Mellangatan -- Slöj

# Define planners

Here, write your code for your planners. Start with the template code for the depth first search and extend.

In [ ]:
def BreadthFirst(num_nodes, mission, f_next, heuristic=None, num_controls=0):
    pass

def Dijkstra(num_nodes, mission, f_next, heuristic=None, num_controls=0):
    pass

def Astar(num_nodes, mission, f_next, heuristic=None, num_controls=0):
    pass

def BestFirst(num_nodes, mission, f_next, heuristic=None, num_controls=0):
    pass

# Define heuristic for Astar and BestFirst planners

Define the heuristic for Astar and BestFirst. The ```LatLongDistance``` function will be useful.

In [ ]:
def cost_to_go(x, xg):
    p_x = osmMap.nodeposition[x]
    p_g = osmMap.nodeposition[xg]
    return 0.0;

# Investigations using all planners